In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from source.read_extrasensory import readMagGyr

In [5]:
X, y, I, Int, features, dimensions, intensities = readMagGyr()

In [6]:
X = X.transpose([0, 2, 1])

In [7]:
def idsStd(ids, X, I):
    D = X.shape[2]
    ind_std = []
    for ind in ids:
        X_indices = np.where(I==ind)
        stdss = []
        for k in range(D):
            dstd = np.std( X[X_indices][:, :, k])
            stdss.append(dstd)
        ind_std.append(stdss)
    ind_std = np.array(ind_std)
    return ind_std


align_x = True
ids_train = np.unique(I)

X_train = X
I_train = I

ind_std_train = idsStd(ids_train , X_train, I_train)
if align_x:
    N_tr, T, D = X_train.shape
    X_train_sh = np.zeros(X_train.shape)
    for i in range(N_tr):
        for k in range(D):
            mag = np.mean(X_train[i, :, k], axis = 0)
            indice = np.where(ids_train ==  I_train[i])[0][0]
            std = ind_std_train[indice][k] * 6
            X_train_sh[i, :, k] = (X_train[i, :, k] - mag) / std

else:
    X_train_sh = np.copy(X_train)

X = X_train_sh

In [9]:
def filter_dimensions(X, all_dimensions, dimensions):
    idx = [np.where(all_dimensions == dim)[0][0] for dim in dimensions]
    return X[:, :, idx]

def folding_v2(X):
    N, T, D = X.shape
    X_f = np.zeros((N, D * T))
    for n in range(N):
        for t in range(T):
            for d in range(D):
                X_f[n, D * t + d] = X[n, t, d]
    return X_f

print(X.shape)

X_f = filter_dimensions(X, dimensions, np.array(['gyr_x', 'gyr_y', 'gyr_z']))
# print(X.shape)
# print(X_f.shape)
# features = folding_v2(X_f)
# print(features.shape)
# X_f = X_f.transpose([0, 2, 1])


(14000, 80, 6)


In [15]:
# X_f = X_f.transpose([0, 2, 1])
X_f.shape

(14000, 80, 3)

In [10]:
# N, D, T = X_f.shape
# for n in range(N):
#     for d in range(D):
#         X_f[n, d, :] = X_f[n, d, :] - X_f[n, d, :].mean()
        
features = folding_v2(X_f)

In [11]:
features.shape

(14000, 240)

In [12]:
import umap

reducer = umap.UMAP(n_components=2, metric='braycurtis')
coords = reducer.fit_transform(features, y=y)

/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [13]:
import sys
sys.path.insert(0, '/home/texs/Documentos/Repositories/mts_viz')

from server.source.storage import MTSStorage

In [16]:
storage = MTSStorage('extraSensory')
storage.load()

participants_map = {ind: str(ind) for ind in np.unique(I)}
intensities_map = {i: intensities[i] for i in range(len(intensities))}


activities = ['LYING_DOWN', 'SITTING', 'STANDING_IN_PLACE', 'STANDING_AND_MOVING', 'WALKING', 'RUNNING', 'BICYCLING']
activities_map = {i: activities[i] for i in range(len(activities))}

storage.add_mts(
    'gyr_mag', X_f, 
    # dimensions, 
    ['gyr_x', 'gyr_y', 'gyr_z'],
    coords={'umap': coords}, 
    labels={'activities': y, 'participants': I, 'intensity': Int}, 
    labelsNames={'activities': activities_map, 'participants': participants_map, 'intensity': intensities_map},
    sampling = True,
    n_samples = 1000
)
storage.save()